In [1]:
import os
import re
import pickle as pkl
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp

from null_importance import get_null_importance


pd.set_option('max_rows', 500, 'max_columns', 200)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

df_row_train['url_sit'] = df_row_train['url'].map(lambda x: x.split('/')[0])
df_row_train['url_page'] = df_row_train['url'].map(lambda x: x.split('/')[1])

df_row_val['url_sit'] = df_row_val['url'].map(lambda x: x.split('/')[0])
df_row_val['url_page'] = df_row_val['url'].map(lambda x: x.split('/')[1])



df_train_info = df_row_train.copy().sort_values(by='op_datetime')

In [4]:
with open(os.path.join(path_new_data, 'no_times_data.pkl'), 'rb') as f:
    df = pkl.load(f)

In [5]:
df_tmp = pd.read_csv('/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/new_data/corpus_txt/models_device_num_transform_embedding.txt', skiprows=1, header=None, sep=' ' )
df_tmp.columns = ['corpus_{}'.format(i) for i in range(6)]
df_tmp

,corpus_0,corpus_1,corpus_2,corpus_3,corpus_4,corpus_5
0,u9diCFdYZ,1.191250,0.884346,0.960100,-0.587945,-0.437067
1,w2CfuqTz3,-0.310487,0.538982,1.129704,-1.143607,0.040968
2,YBCE8ld50,0.031544,1.011728,-0.039371,-1.519553,-0.246772
3,5KbVyNsBf,0.134305,0.212648,0.355266,-1.386184,-0.776587
4,FQND8WWo5,0.745319,0.974483,0.725129,-0.819523,-1.404120
...,...,...,...,...,...,...
3105,4XpDhY9N8,0.447430,0.264440,0.598619,-1.016775,-0.057525
3106,b8PyvUL9p,0.446785,0.555997,0.494677,-0.910030,-0.297620
3107,6CzA6Vd7a,0.249200,0.495751,0.629262,-0.913831,-0.121946
3108,6zzVVzNbn,0.279105,0.324233,0.421833,-1.026533,-0.260341


In [6]:
df = pd.merge(left=df, right=df_tmp, how='left', left_on='url', right_on='corpus_0')

In [7]:
# 删除时间及类别型变量过多的特征
remove_col = ['op_datetime', 'op_month', 'user_name', 
              'op_times_groups', 'system_op_times_groups', 'op_days', 'op_month',
              'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3','corpus_0']

remove_col = [x for x in df.columns if x in remove_col]
df = df.drop(columns=remove_col)
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_type_macos_cumsum,system_system_op_times_groups_os_type_macos_error_code_cumsum,system_system_op_times_groups_os_version_cumunique,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_os_version_win11_cumsum,system_system_op_times_groups_os_version_win11_error_code_cumsum,system_system_op_times_groups_ip_type_cumunique,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_

In [8]:
df_row_train = df[df[y_label].notna()].reset_index(drop=True)
df_row_val = df[df[y_label].isna()].reset_index(drop=True)

df_train, df_test, convert_cols = sp.transform_data_detail(df_row_train, df_row_val, y_label, excel_path=path_output_report)
df_train.head()

sheet05.可能为数值类型的object类型数据统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet06.数据预处理在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


,id,department,browser_version,browser,os_version,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_cumsum,system_system_op_times_groups_http_status_code_200_cumsum,system_system_op_times_groups_op_city_深圳_cumsum,system_system_op_times_groups_op_city_深圳_error_code_cumsum,system_system_op_times_groups_op_city_成都_cumsum,system_system_op_times_groups_op_city_成都_error_code_cumsum,system_system_op_times_groups_op_city_杭州_cumsum,system_system_op_times_groups_op_city_杭州_error_code_cumsum,system_system_op_times_groups_op_city_北京_cumsum,system_system_op_times_groups_op_city_北京_error_code_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_error_code_cumsum,system_system_op_times_groups_url_xxx.com/github_cumsum,system_system_op_times_groups_url_xxx.com/github_error_code_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_error_code_cumsum,system_system_op_times_groups_url_page_github_cumsum,system_system_op_times_groups_url_page_github_error_code_cumsum,system_system_op_times_groups_url_page_download_cumsum,system_system_op_times_groups_url_page_download_error_code_cumsum,system_system_op_times_groups_url_sit_xxx.com_cumsum,system_system_op_times_groups_url_sit_xxx.com_error_code_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_error_code_cumsum,system_op_days_cumsum,system_op_days_error_code_cumsum,system_op_days_department_rd_cumsum,system_op_days_department_rd_error_code_cumsum,system_op_days_department_sales_cumsum,system_op_days_department_sales_error_code_cumsum,system_op_days_browser_version_edge_93_cumsum,system_op_days_browser_version_edge_93_error_code_cumsum,system_op_days_browser_version_chrome_90_cumsum,system_op_days_browser_version_chrome_90_error_code_cumsum,system_op_days_browser_chrome_cumsum,system_op_days_browser_chrome_error_code_cumsum,system_op_days_browser_edge_cumsum,system_op_days_browser_edge_error_code_cumsum,system_op_days_os_type_win_cumsum,system_op_days_os_type_win_error_code_cumsum,system_op_days_os_version_win10_cumsum,system_op_days_os_version_win10_error_code_cumsum,system_op_days_os_version_win7_cumsum,system_op_days_os_version_win7_error_code_cumsum,system_op_days_ip_type_内网_cumsum,system_op_days_ip_type_内网_error_code_cumsum,system_op

In [9]:
df_tmp.columns

Index(['corpus_0', 'corpus_1', 'corpus_2', 'corpus_3', 'corpus_4', 'corpus_5'], dtype='object')

In [10]:
feats = ['day_cos', 'day_sin', 'dayofweek', 'dayofweek_sin', 'hour_cos',
       'op_days_ip_transform_cumsum',
       'op_days_ip_transform_os_type_win_error_code_cumsum',
       'op_days_ip_transform_url_sit_xxx.com_cumsum',
       'op_days_user_name_os_version_win10_error_code_cumsum',
       'op_diff_second1',
       'op_month_ip_transform_http_status_code_200_cumsum',
       'op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum',
       'op_times_groups_device_num_transform_http_status_code_200_cumsum',
       'op_times_groups_user_name_browser_chrome_error_code_cumsum',
       'op_times_groups_user_name_browser_version_edge_93_cumsum',
       'op_times_groups_user_name_error_code_cumsum',
       'op_times_groups_user_name_http_status_code_200_cumsum',
       'op_times_groups_user_name_os_type_win_error_code_cumsum',
       'system_op_days_http_status_code_200_cumsum',
       'system_op_days_op_city_北京_error_code_cumsum',
       'system_op_days_op_city_成都_error_code_cumsum',
       'system_op_days_op_city_深圳_error_code_cumsum',
       'system_op_days_os_version_win7_error_code_cumsum',
       'system_op_diff_second1_log_log',
       'system_system_op_times_groups_browser_version_edge_93_error_code_cumsum',
       'system_system_op_times_groups_http_status_code_200_cumsum',
       'system_system_op_times_groups_op_city_北京_error_code_cumsum',
       'system_system_op_times_groups_op_city_杭州_error_code_cumsum',
       'system_system_op_times_groups_op_city_深圳_error_code_cumsum',
       'system_system_op_times_groups_os_version_win7_error_code_cumsum',
        'corpus_1', 'corpus_2', 'corpus_3', 'corpus_4', 'corpus_5'
        ]
        

In [11]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

In [12]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [13]:
# feats = feats_importance.sort_values('importance', ascending=False)[:30]['name'].values

In [14]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

KeyError: "['corpus_1', 'corpus_2', 'corpus_3', 'corpus_4', 'corpus_5'] not in index"

In [ ]:
feats_importance.sort_values('importance', ascending=False).shape

In [29]:
tmp = feats_importance.sort_values('importance', ascending=False)['name'].values
tmp.sort()
for i in tmp:
    print(i)

corpus_1
corpus_2
corpus_3
corpus_4
corpus_5
day_cos
day_sin
dayofweek
dayofweek_sin
hour_cos
op_days_ip_transform_cumsum
op_days_ip_transform_os_type_win_error_code_cumsum
op_days_ip_transform_url_sit_xxx.com_cumsum
op_days_user_name_os_version_win10_error_code_cumsum
op_diff_second1
op_month_ip_transform_http_status_code_200_cumsum
op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum
op_times_groups_device_num_transform_http_status_code_200_cumsum
op_times_groups_user_name_browser_chrome_error_code_cumsum
op_times_groups_user_name_browser_version_edge_93_cumsum
op_times_groups_user_name_error_code_cumsum
op_times_groups_user_name_http_status_code_200_cumsum
op_times_groups_user_name_os_type_win_error_code_cumsum
system_op_days_http_status_code_200_cumsum
system_op_days_op_city_北京_error_code_cumsum
system_op_days_op_city_成都_error_code_cumsum
system_op_days_op_city_深圳_error_code_cumsum
system_op_days_os_version_win7_error_code_cumsum
system_op_diff_second1_log_log
system_system_op_